In [ ]:
## UNFINISHED ##

In [ ]:
import pandas as pd
import numpy as np
import pylab as p
import matplotlib.pyplot as plot
from collections import Counter
from load_data import load_financial_data, load_sentiment_data
import re

#importing packages for the prediction of time-series data
import statsmodels.api as sm
import statsmodels.tsa.api as smt
import statsmodels.formula.api as smf
from statsmodels.tsa.arima_model import ARIMA


In [ ]:
# Load the data for Bitcoin
df_financials = load_financial_data(filepath = 'data/top20_aggre_price_reduced.csv', start_date = '2017-07-28', end_date = '2018-07-27', tickers = 'BTC')
#df_financials = aggregate_financials(df_financials)

In [ ]:
# Plot the Bitcoin prices to check for Stationarity
plot.plot(df_financials)
plot.xlabel('Date', fontsize=12)
plot.ylabel('Price in USD', fontsize=12)
plot.title("Price distribution of bitcoin", fontsize=15)
plot.show()

In [ ]:
# Test financial values for random walk -> can not reject 0 hypothesis that it is a random walk
from statsmodels.tsa.stattools import adfuller

# Run the ADF test on the price series and print out the results
results = adfuller(df_financials)
print(results)

# Just print out the p-value
print('The p-value of the test on prices is: ' + str(results[1]))

In [ ]:
# Test financial returns for random walk -> can reject 0 hypothesis that it is a random walk
from statsmodels.tsa.stattools import adfuller

# Create a DataFrame of AMZN returns
df_financials_ret = df_financials.pct_change()

# Eliminate the NaN in the first row of returns
df_financials_ret = df_financials_ret.dropna()

# Run the ADF test on the return series and print out the p-value
results = adfuller(df_financials_ret)
print('The p-value of the test on returns is: ' + str(results[1]))

In [ ]:
# Augmented Dicky Fuller Test for Stationarity
from statsmodels.tsa.stattools import adfuller

def test_stationarity(x):


    #Determing rolling statistics
    rolmean = x.rolling(window=22,center=False).mean()

    rolstd = x.rolling(window=12,center=False).std()
    
    #Plot rolling statistics:
    orig = plot.plot(x, color='blue',label='Original')
    mean = plot.plot(rolmean, color='red', label='Rolling Mean')
    std = plot.plot(rolstd, color='black', label = 'Rolling Std')
    plot.legend(loc='best')
    plot.title('Rolling Mean & Standard Deviation')
    plot.show(block=False)
    
    #Perform Dickey Fuller test    
    result=adfuller(x)
    print('ADF Stastistic: %f'%result[0])
    print('p-value: %f'%result[1])
    pvalue=result[1]
    for key,value in result[4].items():
         if result[0]>value:
            print("The graph is non stationery")
            break
         else:
            print("The graph is stationery")
            break;
    print('Critical values:')
    for key,value in result[4].items():
        print('\t%s: %.3f ' % (key, value))


In [ ]:
# Log transform the series to unskew data
ts_log = np.log(df_financials)
plot.plot(ts_log, color = 'green')
plot.show()

test_stationarity(ts_log)

In [ ]:
# Differnciating
ts_log_diff = ts_log - ts_log.shift()
plot.plot(ts_log_diff)
plot.show()

ts_log_diff.dropna(inplace = True)
test_stationarity(ts_log_diff)

In [ ]:
# Autocorrelation
from pandas import Series
from matplotlib import pyplot
from statsmodels.graphics.tsaplots import plot_acf
series = ts_log_diff
plot_acf(series, lags=50)
pyplot.show()

In [ ]:
# Partial Autocorrelation
from pandas import Series
from matplotlib import pyplot
from statsmodels.graphics.tsaplots import plot_pacf
series = ts_log_diff
plot_pacf(series, lags=50)
pyplot.show()

In [ ]:
# Use Auto Regressive model with lag = 4 
# follow lag
model = ARIMA(ts_log, order=(4,1,0))  
results_ARIMA = model.fit(disp=-1)  
plot.plot(ts_log_diff)
plot.plot(results_ARIMA.fittedvalues, color='red')
plot.title('RSS: %.7f'% sum((results_ARIMA.fittedvalues-ts_log_diff)**2))
plot.show()

In [ ]:
# Use Moving Average Model
model = ARIMA(ts_log, order=(0,1,1))  
results_MA = model.fit(disp=-1)  
plot.plot(ts_log_diff)
plot.plot(results_MA.fittedvalues, color='red')
plot.title('RSS: %.7f'% sum((results_MA.fittedvalues-ts_log_diff)**2))
plot.show()

In [ ]:
# Using full ARIMA
from statsmodels.tsa.arima_model import ARIMA
model = ARIMA(ts_log, order=(2,1,0))  
results_ARIMA = model.fit(disp=-1)  
plot.plot(ts_log_diff)
plot.plot(results_ARIMA.fittedvalues, color='red')
plot.title('RSS: %.7f'% sum((results_ARIMA.fittedvalues-ts_log_diff)**2))
plot.show()

In [ ]:
# Use AR model (it performed best)

size = int(len(ts_log)*0.9)
# Divide into train and test
train_arima, test_arima = ts_log[0:size], ts_log[size:len(ts_log)]
history = [x for x in train_arima]
evaluation = pd.DataFrame(index=range(0,len(test_arima)), columns=[['pred', 'exp', 'error %']])
predictions = list()
originals = list()
error_list = list()



print('Printing Predicted vs Expected Values...')
print('\n')
# We go over each value in the test set and then apply ARIMA model and calculate the predicted value. We have the expected value in the test set therefore we calculate the error between predicted and expected value 
for t in range(len(test_arima)):
    model = ARIMA(history, order=(4, 1, 0))
    model_fit = model.fit(disp=-1)
    
    output = model_fit.forecast()
    
    pred_value = output[0]
    
        
    original_value = test_arima[t]
    history.append(original_value)
    
    pred_value = np.exp(pred_value)
    original_value = np.exp(original_value)
    
    # Calculating the error
    error = ((abs(pred_value - original_value)) / original_value) * 100
    
    # Lists for plots
    error_list.append(error)
    predictions.append(float(pred_value))
    originals.append(float(original_value))
    
    # Data Frame for output table
    evaluation.iloc[t][0] = pred_value
    evaluation.iloc[t][1] = original_value
    evaluation.iloc[t][2] = error

    print('predicted = %f,   expected = %f,   error = %f ' % (pred_value, original_value, error), '%')
    

    


In [ ]:
model_fit.summary()

In [ ]:
evaluation.to_csv('evaluation_ARIMA_4_1_0_10_percent_test.csv')

In [ ]:
# After iterating over whole test set the overall mean error is calculated.   
print('\n Mean Error in Predicting Test Case Articles : %f ' % (sum(error_list)/float(len(error_list))), '%')
plot.figure(figsize=(8, 6))
test_day = [t for t in range(len(test_arima))]
labels={'Orginal','Predicted'}
plot.plot(test_day, predictions, color= 'green')
plot.plot(test_day, originals, color = 'orange')
plot.title('Expected Vs Predicted Univariat AR')
plot.xlabel('Hours')
plot.ylabel('Closing Price')
plot.legend(labels)
plot.show()